In [1]:
from __future__ import print_function

import matplotlib.pyplot as plt
import tables as tb
import numpy  as np

%matplotlib inline

Generate some random events in EL plane

In [2]:
def gen_polar_hits(num, N_ELpts, EL_radius):
    """
    Generate hits in polar coordinates. Using polar coords for convenience, since hits must
    be within EL radius
    """
    yp_  = np.empty((num,N_ELpts*2), np.float32)
    yp_[:,0:N_ELpts]         = np.random.uniform(0,EL_radius,(num, N_ELpts))
    yp_[:,N_ELpts:2*N_ELpts] = np.random.uniform(0,2*np.pi,  (num,N_ELpts))
    return yp_

def cartesian_convert(r, theta, N_ELpts):
    """
    Convert polar coords to cartesian
    """
    yc_ = np.empty((r.shape[0],r.shape[1]*2),np.float32)
    for i in range(N_ELpts):
        yc_[:,i]           = np.cos(theta[:,i])*r[:,i] # xcoords
        yc_[:,N_ELpts + i] = np.sin(theta[:,i])*r[:,i] # ycoords
    return yc_

In [3]:
nevts   = 100000
EL_rad  = 198
N_ELpts = 1
coords  = np.zeros((nevts,2),dtype=np.float32)
coords  = np.array([[12.2,35,],[-0.2,.49999999999999999999999999],[-198,-.1]])

polar_coords = gen_polar_hits(nevts, N_ELpts, EL_rad)
coords       = cartesian_convert(polar_coords[:,:N_ELpts],polar_coords[:,N_ELpts:],N_ELpts)

# round for easy extraction from the light table
coords = np.array(np.round(coords),dtype=np.int)

Open light table

In [5]:
f        = tb.open_file('ReproducedFull.h5', 'r')
table    = f.root.Probabilities.data
print(table.description)

Description([('grid_xy', '(2,)f4'), ('sens_id', '(1,)i4'), ('sens_prob', '(1,)f4')])


In [6]:
grid   = np.array(table[:]['grid_xy'  ], dtype=np.int)
probs  = np.array(table[:]['sens_prob'], dtype=np.float32)
ids    = np.array(table[:]['sens_id'  ], dtype=np.int)
id_pos = np.array(f.root.Sensors.XY[12:])

# Note: table.where does not support condition with multidim col (and cant simply index into multidim col)
#test = np.array([[row['sens_id'],row['probs']] for row in table.where("grid_xy[0] == coords[0,0] & grid_xy[1] == coords[0,1]")])

# so, replace ids with positions **takes 30 seconds**
pos = np.ones((len(ids),2), dtype=np.int)*-9999 # -9999 will correspond to PMTs
for p in id_pos: pos[np.where(ids == p[0])[0]] = p[1:]

Generate SiPM maps

In [ ]:
maps   = np.zeros((nevts,48,48),dtype=np.float32)
# dictionaries to move to, from sipm position <---> index ##make faster
ind2pos = {}
pos2ind = {}
for i in range(48):
    pos2ind[-235 + 10*i] = i
    ind2pos[i] = -235 + 10*i

# for each time slice: collect non zero sipm responses, populate maps 
for z,xy in enumerate(coords):
    
    # collect non zero sipm responses
    i         = np.where((grid == xy).all(axis=1))[0]
    sli_pos   = pos[i]
    sli_probs = probs[i]
    
    # populate sipm maps for this time slice
    for posx,posy,probj in zip(sli_pos[:,0], sli_pos[:,1], sli_probs): 
        if posx != -9999:
            xidx = pos2ind[posx]
            yidx = pos2ind[posy]
            maps[z,xidx,yidx] = probj
    if (z+1)%1000==0: print(str(z/float(nevts)) + ' complete...')

In [7]:
testmaps   = np.zeros((nevts,48,48),dtype=np.float32)

# for each time slice: collect non zero sipm responses, populate maps 
for z,xy in enumerate(coords):
    
    # collect non zero sipm responses
    i         = np.where((grid == xy).all(axis=1))[0]
    sli_pos   = pos[i]
    sli_idx   = np.array((sli_pos + 235) / 10, dtype=np.int)
    sli_probs = probs[i]
    #testmaps[z,sli_idx[:,0],sli_idx[:,1]] = sli_probs
    
    # populate sipm maps for this time slice
    for xidx,yidx,probj in zip(sli_idx[:,0], sli_idx[:,1], sli_probs): 
        if xidx != -977: testmaps[z,xidx,yidx] = probj
    if (z+1)%1000==0: print(str(z/float(nevts)) + ' complete...')

0.00999 complete...
0.01999 complete...
0.02999 complete...
0.03999 complete...
0.04999 complete...
0.05999 complete...
0.06999 complete...
0.07999 complete...
0.08999 complete...
0.09999 complete...
0.10999 complete...
0.11999 complete...
0.12999 complete...
0.13999 complete...
0.14999 complete...
0.15999 complete...
0.16999 complete...
0.17999 complete...
0.18999 complete...
0.19999 complete...
0.20999 complete...
0.21999 complete...
0.22999 complete...
0.23999 complete...
0.24999 complete...
0.25999 complete...
0.26999 complete...
0.27999 complete...
0.28999 complete...
0.29999 complete...
0.30999 complete...
0.31999 complete...
0.32999 complete...
0.33999 complete...
0.34999 complete...
0.35999 complete...
0.36999 complete...
0.37999 complete...
0.38999 complete...
0.39999 complete...
0.40999 complete...
0.41999 complete...
0.42999 complete...
0.43999 complete...
0.44999 complete...
0.45999 complete...
0.46999 complete...
0.47999 complete...
0.48999 complete...
0.49999 complete...


Save the coordinate of the randomly generated hits, and their SiPM maps

In [1]:
f = tb.open_file('NEW_krypton_maps_100000.h', 'w')
filters = tb.Filters(complib='blosc', complevel=9, shuffle=False)

atom    = tb.Atom.from_dtype(maps.dtype)
tmaps   = f.create_earray(f.root, 'maps',   atom, (0,48,48), filters=filters) 

atom    = tb.Atom.from_dtype(coords.dtype)
tcoords = f.create_earray(f.root, 'coords', atom, (0,2),     filters=filters)

for i in range(nevts):
    tmaps.append([testmaps[i]])
    tcoords.append([testcoords[i]])
print(f)
f.close()

NameError: name 'tb' is not defined